## ***ΕΠΕΞΕΡΓΑΣΙΑ ΦΥΣΙΚΗΣ ΓΛΩΣΣΑΣ ΚΑΙ ΣΗΜΑΣΙΟΛΟΓΙΚΟΣ ΙΣΤΟΣ***
ΕΡΓΑΣIΑ ΕΞΑΜΝΟΥ 2024-2025 <br>
***Κωνσταντίνος Ιωάννης Σιατερλής 18390192***

Το πρόβλημα είναι να μπορέσει μια μηχανή να αναγνώριση αν ένα κείμενο είναι spam η όχι 

Αρχικά με την βοήθεια της βιβλιοθήκης numpy διαβάζω τα δεδομένα με καταλαλητά κωδικοποίηση. Η numpy διαβάζει όλα τα αλφαριθμητικά σαν ένα κομματικά με την χρήση του quotechar στο numpy.loadtxt, επίσης αρχικοποιώ μερικές μεταβλητές για ευκολία.


In [1]:
import numpy as np
import nltk
from sklearn.metrics import classification_report

DATA = np.loadtxt("./mail_data.csv", delimiter=",", dtype=str, usecols=(0,1), encoding ='utf-8', quotechar='"', skiprows=1)
#np.random.shuffle(DATA)

SPAMMAP = {'ham': 0, 'spam': 1}
STOPWORDS = nltk.corpus.stopwords.words("english")

TRAINSIZE = 0.2 

print(DATA[:5])

[['ham'
  'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...']
 ['ham' 'Ok lar... Joking wif u oni...']
 ['spam'
  "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"]
 ['ham' 'U dun say so early hor... U c already then say...']
 ['ham' "Nah I don't think he goes to usf, he lives around here though"]]


# **Preperation**

Για να μπορέσω αναγνωρίσω αν ειναι spam η όχι  πρώτα πρέπει να επεξεργαστώ της προτάσεις. Η συντήρηση preparation έχει δημιουργηθεί για αυτόν τον λόγο δηλαδή :

* Κανονικοποίηση Κειμένου
* Διαίρεση σε Λέξεις
* Αφαίρεση Λέξεων
* Ανάλυση Μορφολογίας
* Μέρος του Λόγου

Επιστρέφει ενα bag of words

In [2]:
def preparation(str_, stopWords):
    finalList =[]
    ps = nltk.stem.PorterStemmer()

    str_.lower()
    str_ = str_.replace("-", " ")

    finalList = nltk.tokenize.word_tokenize(str_)
    finalList = ["".join( ch for ch in w if ch.isalnum() ) for w in finalList if w not in stopWords]
    finalList = [ps.stem(w) for w in finalList ]
    finalList = [w for w in finalList if w != ""]

    tags = nltk.pos_tag(finalList)
    finalFeatures = {}

    # Bag the words
    for word, tag in tags:
        finalFeatures[f'{word}_{tag}'] = True

    return finalFeatures

finalDataSet = []

for i in DATA:
    finalDataSet.append((preparation(i[1], STOPWORDS), i[0]))

finalDataSet[0]

({'go_VB': True,
  'jurong_JJ': True,
  'point_NN': True,
  'crazi_NN': True,
  'avail_NN': True,
  'bugi_NN': True,
  'n_RB': True,
  'great_JJ': True,
  'world_NN': True,
  'la_NN': True,
  'e_VBP': True,
  'buffet_JJ': True,
  'cine_NN': True,
  'got_VBD': True,
  'amor_JJ': True,
  'wat_NN': True},
 'ham')

Για την εκπαίδευση του αλγόριθμου πρέπει να χωρίσω 
τα δεδομένα σε εκπαίδευσης και δοκιμής.

Με το len περνώ το 20% του του τέλους και το 80% της αρχής

In [3]:
maxSizeSentence = max(len(x[0]) for x in finalDataSet)
sizePerSentence = [len(x[0]) for x in finalDataSet]
avgSizeSentence = int(sum(sizePerSentence) / len(sizePerSentence))

# train test split, 80 - 20
trainData = finalDataSet[:(int(len(finalDataSet) * TRAINSIZE))]
testData = finalDataSet[(int(len(finalDataSet) * TRAINSIZE)):]

print("Length of training data :", len(trainData))
print("Length of test data :", len(testData), "\n")

Length of training data : 1114
Length of test data : 4458 



# **Naive Bayes**
Ο Naive Bayes είναι ένας πιθανοκρατικός αλγόριθμος μηχανικής μάθησης βασισμένος στο Θεώρημα του Bayes ο οποίος κάνει μια υπόθεση ότι όλα τα χαρακτηριστικά είναι ανεξάρτητα μεταξύ τους δεδομένης της κατηγορίας.

Αλγόριθμος naive bayes υπάρχει έτυμος στην βιβλιοθήκη NLTK οπου παίρνει σαν είσοδο ένα πινάκα από bag of words.
Υστερα προβλέπει τα δεδομένα εκπαίδευσης αν είναι spam η όχι και τα βάζει μέσα σε ένα πίνακα.

Με την classification_report (sklearn.metrics) βγάζει μια αναφορά ακρίβειας (F1-score)

In [4]:
testNativeBytes = nltk.NaiveBayesClassifier.train(trainData)

yPred = [testNativeBytes.classify(x[0]) for x in testData]
yTest = [x[1] for x in testData]

print(classification_report(yTest, yPred))

              precision    recall  f1-score   support

         ham       1.00      0.83      0.91      3879
        spam       0.47      0.97      0.63       579

    accuracy                           0.85      4458
   macro avg       0.73      0.90      0.77      4458
weighted avg       0.93      0.85      0.87      4458



Ο αλγόριθμος Naive Bayes είναι ιδιαίτερα αποδοτικός στην κατηγοριοποίηση κειμένου. Λειτουργεί καλά με πολλά δεδομένα είναι γρήγορος στην εκπαίδευση και πρόβλεψη. Παρά την απλότητά του έχει αρκετούς περιορισμούς υποθέτει την ανεξαρτησίας μεταξύ των χαρακτηριστικών η οποία σπάνια ισχύει.

# **Word To Vector και Logistic Regression**

Ο Word2Vec είναι ένα αλγόριθμο νευρωνικού δίκτυου που εκπαιδεύεται έτσι ώστε οι λέξεις που εμφανίζονται με παρόμοια συμφραζόμενα να αποκτούν παρόμοια διανύσματα. 


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import gensim

scamOrNot = np.array([mail[1] for mail in finalDataSet])

tmpSentences = [list(mail[0].keys()) for mail in finalDataSet]
tmpSentences[:2]

[['go_VB',
  'jurong_JJ',
  'point_NN',
  'crazi_NN',
  'avail_NN',
  'bugi_NN',
  'n_RB',
  'great_JJ',
  'world_NN',
  'la_NN',
  'e_VBP',
  'buffet_JJ',
  'cine_NN',
  'got_VBD',
  'amor_JJ',
  'wat_NN'],
 ['ok_JJ', 'lar_JJ', 'joke_NN', 'wif_NN', 'u_JJ', 'oni_NN']]

Αλγόριθμος word2vec υπάρχει έτυμος στην βιβλιοθήκη gensim, περνει σαν εισοδο τις προτασεις και τα μεγεθει του vector που το καθε ενα απο αυτα θα χαρακτηρίζει μια λέξει.

Η συνάρτηση WordsToVec μετατρέπει μια πρόταση σε ένα σύνολα από πίνακες με τα αντίστοιχα vectors των λέξεων αυτής της πρότασης.

In [6]:
def WordsToVec(mail):
    tmpVec = [model.wv[word] for word in mail if word in model.wv]
    if len(tmpVec) != 0:
        return tmpVec
    return []

# Montelo Word2vercor
model = gensim.models.Word2Vec(sentences=tmpSentences, vector_size=100, window=5, min_count=1, workers=4)

# Lejeis se areiumoys
wordVecNumbers = [WordsToVec(mail) for mail in tmpSentences]
mymax = max(len(x) for x in wordVecNumbers)

Για να αναγνωριστούν η πρότασης που είναι spam χρησιμοποιείται η Logistic Regression, αλλά η  Logistic Regression παίρνει σαν είσοδο ένα πινάκα δυο άξονες, αν είναι spam η όχι και το κείμενο, για αυτό τον λόγο ενώνω όλα τα vector των λέξεων σε ένα. Τελος βγάζει μια αναφορά ακρίβειας (F1-score) 

In [7]:
# προσθηκη κενών για ομογένεια
for x in wordVecNumbers:
    x += [[0] * 100 for _ in range(mymax - len(x))]

# Μεταφραση του πινακα σε numpy
wordVecNumbers = np.array(wordVecNumbers)

# Μετατροπη του πινακα σε 2 άξονες
wordVecNumbers = wordVecNumbers.reshape(wordVecNumbers.shape[0], -1)

# 80 - 20
XTrain, XTest, yTrain, yTest = train_test_split(wordVecNumbers, scamOrNot, test_size=TRAINSIZE)

# LogisticRegression
clf = LogisticRegression()
clf.fit(XTrain, yTrain)

yPred = clf.predict(XTest)
print(classification_report(yTest, yPred))

              precision    recall  f1-score   support

         ham       0.90      0.98      0.94       972
        spam       0.68      0.29      0.41       143

    accuracy                           0.89      1115
   macro avg       0.79      0.64      0.68      1115
weighted avg       0.88      0.89      0.87      1115



Ο συνδυασμός Word2Vec + Classifier είναι μια πολύ ισχυρή προσέγγιση για κατανόηση και κατηγοριοποίηση κειμένου. Το Word2Vec μετατρέπει τις λέξεις σε σημασιολογικά πλούσια διανύσματα που συλλαμβάνουν σχέσεις μεταξύ λέξεων ενώ ένας Classifier κατηγοριοποιεί τα δεδομένα με βάση αυτά τα διανύσματα. Ωστόσο ο Word2Vec παράγει ένα σταθερό διάνυσμα ανά λέξη ανεξαρτήτως του πλαισίου πχ "τράπεζα αίματος" είτε για "τράπεζα χρήματος" και δεν λαμβάνει υπόψη τη συντακτική σειρά ή τη γραμματική της πρότασης 

# **Recurrent Neural Networks - RNNs**
Ένα επαναλαμβανόμενο νευρωνικό δίκτυο είναι ένα μοντέλο μάθησης που έχει εκπαιδευτεί να επεξεργάζεται και να μετατρέπει μια διαδοχική είσοδο δεδομένων σε μια συγκεκριμένη διαδοχική έξοδο δεδομένων.

Η pyporch έχει όλα τα απαραίτητα εργαλεία για να ένα RNN, πρώτα ώμος χρειάζεται ένα λεξικό και επειδή δεν μπορεί να καταλάβει λέξη τις χαρακτηρίζονται απο ένα άκαιρος.

In [8]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

#maxSizeSentence avgSizeSentence
SIZEOFRNNS = avgSizeSentence

# vocabulary
allFeatures = set()
for features, _ in trainData:
    allFeatures.update(features.keys())

# +1 to reserve 0 for padding
featureAndIdx = {word: i + 1 for i, word in enumerate(sorted(allFeatures))}
vocSize = len(featureAndIdx) + 1

Για να μπόρεση να αναγνώριση τα δεδομένα ένας αλγόριθμος RNN πρέπει να φταίχτη ένας συγκεκριμένος τύπος δεδομένον, που είναι παιδί του Dataset αντικείμενου από την pytorch.

Η encFeatures αλλάζει τις λέξις με τα αντίστοιχα αναγνωριστικά τους και προσθετή κενά για ομογένεια.

In [9]:
def encFeatures(featureDict, maxLen = SIZEOFRNNS):
    indices = [featureAndIdx[k] for k in featureDict if k in featureAndIdx]
    indices = indices[:maxLen]
    indices += [0] * (maxLen - len(indices))
    return indices

# DataSet RNNS
class myDataset(Dataset):
    def __init__(self, data, maxLen = SIZEOFRNNS):
        self.X = [encFeatures(x, maxLen) for x, _ in data]
        self.y = [SPAMMAP[y] for _, y in data]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx]), torch.tensor(self.y[idx], dtype=torch.float)

Το Νευρωνικό δίκτυο είναι ένα βασικό δίκτυο RNNs με νευρώνες εισόδου όσο το μέσο όρο των λέξεων στων προτάσεων, κρυφά επίπεδα 16 και έξοδο μια γραμμικού τύπο συνάρτηση με μέγιστο το 1

In [10]:
class RNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = self.embed(x)
        _, h = self.rnn(x)
        out = self.fc(h.squeeze(0))
        return torch.sigmoid(out)

Πέρασμα των δεδομένον εκπαίδευσης και αρχικοποιήσει μεταβλητών για διόρθωση σφάλματος οπός το κριτήριο σφάλματος (criterion) και ο ρυθμός μάθησης του αλγορίθμου (optimizer)

In [13]:
# Train
print(SIZEOFRNNS)
dataset = myDataset(trainData)
loader = DataLoader(dataset, batch_size=1, shuffle=True)

model = RNNClassifier(vocab_size=vocSize, embed_size=SIZEOFRNNS, hidden_size=16)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(4):
    for x, y in loader:
        yPred = model(x).squeeze().unsqueeze(0)
        loss = criterion(yPred, y)

        # Clean
        optimizer.zero_grad()
        # Computes the loss
        loss.backward()
        # Updates the weights
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


10
Epoch 1, Loss: 0.1298
Epoch 2, Loss: 2.3247
Epoch 3, Loss: 0.0859
Epoch 4, Loss: 0.0052


Πρόβλεψη των προτάσεων μεσώ μια συντήρησης και βγάζει μια αναφορά ακρίβειας (F1-score) 

In [14]:
def predict(featureDict):
    x = torch.tensor(encFeatures(featureDict)).unsqueeze(0)
    with torch.no_grad():
        out = model(x)
    return 'spam' if out.item() > 0.5 else 'ham'

yPred = [ predict(x[0]) for x in testData ]
yTest = [ str(x[1]) for x in testData ]

print("\n", classification_report(yTest, yPred))


               precision    recall  f1-score   support

         ham       0.93      0.98      0.95      3879
        spam       0.78      0.49      0.60       579

    accuracy                           0.92      4458
   macro avg       0.86      0.73      0.78      4458
weighted avg       0.91      0.92      0.91      4458



Τα RNNs είναι ιδιαίτερα κατάλληλα για σειριακά δεδομένα όπως κείμενο επειδή έχουν “μνήμη” λαμβάνουν υπόψη τους προηγούμενες λέξεις. Αυτό τα καθιστά ισχυρά για εργασίες όπως ανάλυση συναισθήματος και κατηγοριοποίηση προτάσεων, όπου η σειρά των λέξεων έχει σημασία. Ωστόσο, παρουσιάζουν σημαντικά αρνητικά, όπως το πρόβλημα της έκρηξης των βαθμίδων (vanishing/exploding gradients), που τα κάνει δύσκολα στην εκπαίδευση σε μεγάλες ακολουθίες.